<a href="https://colab.research.google.com/github/khamies/Piano-VAE/blob/master/play_with_model.ipynb" 
target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Download data and files.
!git clone https://github.com/Khamies/Piano-VAE.git
import os 
os.chdir("Piano-VAE")

Cloning into 'Piano-VAE'...
Updating files:  81% (888/1096)
Updating files:  82% (899/1096)
Updating files:  83% (910/1096)
Updating files:  84% (921/1096)
Updating files:  85% (932/1096)
Updating files:  86% (943/1096)
Updating files:  87% (954/1096)
Updating files:  88% (965/1096)
Updating files:  89% (976/1096)
Updating files:  90% (987/1096)
Updating files:  91% (998/1096)
Updating files:  92% (1009/1096)
Updating files:  93% (1020/1096)
Updating files:  94% (1031/1096)
Updating files:  95% (1042/1096)
Updating files:  96% (1053/1096)
Updating files:  97% (1064/1096)
Updating files:  97% (1068/1096)
Updating files:  98% (1075/1096)
Updating files:  99% (1086/1096)
Updating files: 100% (1096/1096)
Updating files: 100% (1096/1096), done.


In [2]:
import torch
from model import LSTM_MUSIC_VAE
from dataset import PianoGenerationDataset
from utils.utils import interpolate

In [3]:
from utils.midi.midi_utils import midiwrite

In [4]:
# Settings

torch.manual_seed(1000)
device = "cuda" if torch.cuda.is_available() else "cpu"

batch_size = 32
bptt = 60
lr = 0.001

embed_size = 300
hidden_size = 256
latent_size = 16
lstm_layer=1
notes_size = 88 #unique piano notes

## Load the data

In [5]:
trainset = PianoGenerationDataset('./data/Nottingham/train/', longest_sequence_length=None)
testset = PianoGenerationDataset('./data/Nottingham/test/', longest_sequence_length=None)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True, drop_last=True)

## Load the model

In [7]:
vocab_size = notes_size
model = LSTM_MUSIC_VAE(vocab_size = vocab_size, embed_size = embed_size, hidden_size = hidden_size, latent_size = latent_size).to(device)


checkpoint = torch.load("models/LSTM_Music_VAE.pt", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint["model"])

<All keys matched successfully>

##Sample Generation

In [8]:
# Use pitch_index to set the note value (pitch) for the initial note.

z = torch.randn(1,1,latent_size).to(device)
sample = model.inference(50,z)
midiwrite("sample.midi",sample) 

# Interpolation

In [10]:
samples, tone1, tone2 = interpolate(model, n_interpolations= 32, sequence_length= 200)
midiwrite("./tone1.midi",tone1)
midiwrite("./tone2.midi",tone2)
midiwrite("./sample_inter.midi",samples[1])
midiwrite("./sample_inter_2th.midi",samples[4])